# Project 2: Image Stitcher
## Assigned: 02.01.2019
## Due Date: TBD (probably 02.20.2019)

Panoramic photography is ubiquitous, with nearly every digital camera having a mode dedicated to doing it.  Here's an example from the Italian Alps:
<img src="pano.jpg">
Note the extreme aspect ratio: much larger than the 4:3 or 3:2 that is typical of most cameras; suffice to say, the camera that stook this picture did not have a sensor that was this wide.  So how are these things made?  Stated simply, multiple images are taken, mutually identifiable points are located in each of these images, and the images are warped such that these points are coincident.  The matching stage might look like this:
<img src="office.jpeg">

For this project, you will code your own image stitcher from scratch.  Despite the conceptual simplicity of this operation, there are a surprising number of challenges that need to be addressed.  A general framework for a stitcher might look like this:

In [75]:
import cornermatching as cm
import skimage.transform as skt

class Stitcher(object):
    def __init__(self,image_1,image_2):
        self.images = [image_1,image_2]
        
    def find_keypoints(self):
        
        # Guass kernel for convolution
        g_kernal = cm.gauss_kernal(5,2)
        
        # Calculate the harris response of each convolution of I1, I2
        H1 = cm.harris_response(cm.convolve(self.images[0], g_kernal))
        H2 = cm.harris_response(cm.convolve(self.images[1], g_kernal))
        
        # Extract the keypoints from H1, H2 via non-maximal sup
        key_pts_I1 = cm.nonmaxsup(H1)
        key_pts_I2 = cm.nonmaxsup(H2)
        
        # Return the keypoints of I1, I2
        return key_pts_I1, key_pts_I2
    
    def generate_descriptors(self):
        
        # Get the keypoints to generate descriptors from
        key_pts_I1, key_pts_I2 = self.find_keypoints()
        
        # Get descriptors for I1, I2
        des_I1 = cm.descriptorExtractor(self.images[0], key_pts_I1)
        des_I2 = cm.descriptorExtractor(self.images[1], key_pts_I2)
        
        return des_I1, des_I2
        """
        Step 2: After identifying relevant keypoints, we need to come up with a quantitative description of the 
        neighborhood of that keypoint, so that we can match it to keypoints in other images.
        """
        
    def match_keypoints(self):
        
        des_I1, des_I2 = self.generate_descriptors()
        
        best_matches = cm.get_best_matches(des_I1, des_I2)
        secondbest_matches = cm.get_secondbest_matches(des_I1, des_I2, best_matches)
        
        
        filtered_matches = cm.filter_matches(best_matches, secondbest_matches)
        
        return filtered_matches, des_I1
    
        """
        Step 3: Compare keypoint descriptions between images, identify potential matches, and filter likely
        mismatches
        """
        
    def find_homography(self):
        
        # Get the matches between the two images
        matches, des_I1 = self.match_keypoints()
        
        # Now get the coordinates from the matches for RANSAC
        match_coords = []
        for match in matches: #filtered_matches:
    
            match_I1_x = des_I1[match[0]][1]

            match_I1_y = des_I1[match[0]][2]
    
            match_I2_x = match[1][1]
    
            match_I2_y = match[1][2]
    
            match_coords.append([match_I1_x,match_I1_y,match_I2_x,match_I2_y])
        
        # params needed for RANSAC
        print(len(match_coords))
        num_iters = 1000
        r = 3
        d = 4
        n = int(len(matches)/2)
        
        H_best = cm.RANSAC(num_iters, match_coords, n, r, d)
        
        return H_best
        """
        Step 4: Find a linear transformation (of various complexities) that maps pixels from the second image to 
        pixels in the first image
        """
      
    def stitch(self,H):
        # Create a projective transform based on the homography matrix $H$
        proj_trans = skt.ProjectiveTransform(H)
        I1 = self.images[0]
        I2 = self.images[1]
        # Warp the image into image 1's coordinate system
        image_2_transformed = skt.warp(I2,proj_trans)
        I1 = I1 + image_2_transformed
        print(image_2_transformed.shape)
        plt.imshow(image_2_transformed, cmap="gray")
        plt.show()
        """
        Step 5: Transform second image into local coordinate system of first image, and (perhaps) perform blending
        to avoid obvious seams between images.
        """

We will populate these functions over the next several weeks, a process that will involve delving into some of the most elementary operations in digital signal processing.  

As a test case, apply your stitcher to at least four overlapping images that you've taken.  With a stitcher that works on two images, more images can be added by applying the method recursively.

In [ ]:
import matplotlib.pyplot as plt
#def stitchImages():
'''Returns the stiched images recursively'''

images = [plt.imread('im1.jpg').mean(axis=2), plt.imread('im2.jpg').mean(axis=2), plt.imread('im3.jpg').mean(axis=2), plt.imread('im4.jpg').mean(axis=2)]

image_stitcher = Stitcher(images[0], images[1])



In [ ]:
filtered_matches, best_matches = image_stitcher.match_keypoints()

In [ ]:
H,inliers = image_stitcher.find_homography()

In [ ]:

image_stitcher.stitch(H)

In [ ]:
import numpy as np
h = int(len(images[0]))
w = int(len(images[0][0])*2)
s = (h,w)
print(s)
new_image = np.zeros(s)
new_image[0:h, 0:int(w/2)] = images[2]
new_image[0:h, int(w/2):] = images[3]
plt.imshow(new_image, cmap="gray")

for match in filtered_matches:
    x1 = best_matches[match[0]][1]
    y1 = best_matches[match[0]][2]
    x2 = match[1][1]
    y2 = match[1][2]
    print(x1,y1,x2,y2)
    plt.plot([y1,y2+int(w/2)], [x1,x2], color="blue", marker="x")
plt.show()

In [ ]:
print(H)